# ニューラルネットーワークに基づく(実数)MIMO検出

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wadayama/MIKA2019/blob/master/MIMO.ipynb)

本ノートブックでは、ニューラルネットワークにより実数体上のMIMO検出問題を行う。概要は次のとおり。
* $H \in \mathbb{R}^{4 \times 4}$: 干渉行列 (各要素は平均0, 分散1のガウス分布に従う)
* $y = x H + w$: 受信ベクトル(行ベクトル)
* 注意: テキスト(デフォルトで列ベクトル)とは異なっており、行と列が入れ替わっている。本ノートの中ではベクトルはデフォルトで行ベクトル
* $x \in \{+1, -1 \}^4$
* $w \in \mathbb{R}^4$: 各要素が平均0、分散$\sigma^2$のガウス分布に従う乱数ベクトル
* 目標は、受信ベクトルである$y$から送信ベクトル$x$を可能な限り正しく推定すること

## 必要なパッケージのインポート

In [1]:
import torch # テンソル計算など
import torch.nn as nn  # ネットワーク構築用
import torch.optim as optim  # 最適化関数

## グローバル定数の設定

In [2]:
mbs = 5 # ミニバッチサイズ
noise_std = 0.5 # 通信路において重畳される加法的白色ガウス雑音の標準偏差 (\sigma)
n = 4 # アンテナ数
h = 30 # 隠れ層のユニット数
H = torch.normal(mean=torch.zeros(n, n), std=1.0) # 干渉行列
adam_lr = 0.001 # Adamの学習率

## 干渉行列の確認

In [3]:
print(H)

tensor([[-0.6901,  1.6857, -0.6441, -0.4393],
        [-1.8097,  0.7991, -0.8389,  1.1212],
        [-1.5226,  1.5627,  0.6609, -0.3418],
        [-0.3782, -0.7504, -0.1026,  0.5203]])


## ネットワークの定義

In [4]:
class Net(nn.Module): # nn.Module を継承
    def __init__(self): # コンストラクタ
        super(Net, self).__init__()
        self.detector = nn.Sequential(
            nn.Linear(n, h),  # W_1, b_1,
            nn.ReLU(), # 活性化関数としてReLUを利用
            nn.Linear(h, h), # W_2, b_2
            nn.ReLU(),
            nn.Linear(h, n)  # W_3, b_3
        )
    def forward(self, x): # 推論計算をforwardに書く
        x = self.detector(x)
        x = torch.tanh(x) # x \in {+1,-1}^4 なので、最終層はtanhを利用
        return x

## ミニバッチ生成関数

In [5]:
def gen_minibatch():
    x = 1.0 - 2.0 * torch.randint(0, 2, (mbs, n)) # 送信ベクトル x をランダムに生成
    x = x.float()
    w = torch.normal(mean=torch.zeros(mbs, n), std = noise_std) # 加法的白色ガウス雑音の生成
    y = x @ H.t() + w # @は行列ベクトルの積
    return x, y

## ミニバッチ生成関数の実行例

In [6]:
x, y = gen_minibatch()
print('x = ', x)
print('y = ', y)

x =  tensor([[ 1.,  1., -1.,  1.],
        [-1., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [-1., -1., -1.,  1.],
        [ 1., -1., -1.,  1.]])
y =  tensor([[ 0.6164,  0.5239, -1.1730, -0.4618],
        [-1.7533,  1.1149,  0.4016,  1.7341],
        [ 0.2989, -1.2589,  0.4373,  0.2389],
        [-0.4521,  2.6848, -0.7461,  2.2843],
        [-1.2298, -1.3321, -3.8048,  1.0535]])


## 訓練ループ

In [7]:
model = Net() # ネットワークインスタンス生成
loss_func = nn.MSELoss() # 損失関数の指定(二乗損失関数)
optimizer = optim.Adam(model.parameters(), lr=adam_lr) # オプティマイザの指定(Adamを利用)
for i in range(10000):
    x, y = gen_minibatch() # ミニバッチの生成
    optimizer.zero_grad()  # オプティマイザの勾配情報初期化
    estimate = model(y)  # 推論計算
    loss = loss_func(x, estimate)  # 損失値の計算
    loss.backward()  # 誤差逆伝播法(後ろ向き計算の実行)
    optimizer.step()  # 学習可能パラメータの更新
    if i % 1000 == 0:
        print('i =', i, 'loss =', loss.item())

i = 0 loss = 1.1574251651763916
i = 1000 loss = 0.028530091047286987
i = 2000 loss = 0.004815287422388792
i = 3000 loss = 0.012845365330576897
i = 4000 loss = 0.019387107342481613
i = 5000 loss = 0.006736423820257187
i = 6000 loss = 0.0036912679206579924
i = 7000 loss = 0.07159878313541412
i = 8000 loss = 0.0018119346350431442
i = 9000 loss = 0.01168894674628973


## 学習結果の確認

In [8]:
mbs = 1
x, y = gen_minibatch()
print('x = ', x)
print('y = ', y)
x_hat = model(y)
print('x_hat = ', x_hat)

x =  tensor([[-1., -1.,  1.,  1.]])
y =  tensor([[-1.0497,  0.9863, -0.0947,  1.1771]])
x_hat =  tensor([[-0.9936, -0.9972,  0.5932,  0.5281]], grad_fn=<TanhBackward>)


## ゼロフォーシング等化を試す

In [12]:
Hinv = torch.inverse(H.t())
x_hat_zero = y @ Hinv
print('x_hat_zero = ', x_hat_zero)

x_hat_zero =  tensor([[ 0.6752, -1.2395, -1.5136, -1.7360]])


## シンボル誤り率を測定する (ニューラル検出器)

In [10]:
total_syms = 0
error_syms = 0
num_loops  = 1000
mbs = 1
with torch.no_grad():
    for i in range(num_loops):
        x, y = gen_minibatch()
        x_hat = model(y)
        total_syms += n
        error_syms += (torch.sign(x_hat) != x).sum().item()
print('total_syms = ', total_syms)
print('error_syms = ', error_syms)
print('error prob = ', error_syms/total_syms)

total_syms =  4000
error_syms =  64
error prob =  0.016


## シンボル誤り率を測定する(ZF検出器)

In [13]:
total_syms = 0
error_syms = 0
num_loops  = 1000
mbs = 1
Hinv = torch.inverse(H.t())
for i in range(num_loops):
    x, y = gen_minibatch()
    x_hat = y @ Hinv
    total_syms += n
    error_syms += (torch.sign(x_hat) != x).sum().item()
print('total_syms = ', total_syms)
print('error_syms = ', error_syms)
print('error prob = ', error_syms/total_syms)

total_syms =  4000
error_syms =  776
error prob =  0.194
